In [8]:
import requests
from bs4 import BeautifulSoup

In [6]:
def get_daum_news_title(news_id):
    url = 'https://news.v.daum.net/v/{}'.format(news_id)
    resp = requests.get(url)
    
    soup = BeautifulSoup(resp.text)
    
    title_tag = soup.select_one('h3.tit_view')
    if title_tag:
        return title_tag.get_text()
    return""

In [7]:
get_daum_news_title(20210721104401263)

"'댓글 조작' 김경수 징역 2년 확정..지사직 박탈"

In [12]:
def get_daum_news_title(news_id):
    url = 'https://news.v.daum.net/v/{}'.format(news_id)
    resp = requests.get(url)
    
    soup = BeautifulSoup(resp.text)
    
    content = ''
    for p in soup.select('div#harmonyContainer p'):
        content += p.get_text()
        
    return content

In [13]:
get_daum_news_title(20190728165812603)

'[아시아경제 이민우 기자] 일론 머스크 테슬라 최고경영자(CEO)가 자사 전기 자동차 모델에 넷플릭스와 유튜브 등 온라인동영상서비스(OTT)를 탑재할 것이라고 예고했다. 단순히 자율 주행 전기차가 단순히 주행을 위한 정보를 알려주는 것을 넘어 각종 영상 콘텐츠를 즐기는 공간으로도 확장하겠다는 전략으로 풀이된다.27일(현지시간) 더버지 등 주요 외신들에 따르면 머스크 CEO는 자신의 트위터를 통해 이 같은 계획을 밝혔다. 그는 "자동차가 정차했을 때 넷플릭스와 유튜브를 감상할 수 있는 기능이 조만간 추가될 것"이라며 "편안한 좌석과 서라운드 사운드 오디오를 통해 영화관과 같은 느낌을 받을 수 있을 것"이라고 강조했다.테슬라가 이처럼 콘텐츠 방면으로 확장하려 든 것은 이번이 처음이 아니다. 지난달 세계 최대 게임쇼 E3에서는 이미 운전자가 \'폴아웃 쉘터\'라는 게임을 할 수 있을 것이라고 발표한 바 있다. 이후에도 최근 게임업체 아타리사(社)의 자동차 경주 게임 ‘폴포지션’, 슈팅게임 ‘템페스트’, ‘미사일커맨드’ 등 고전 게임을 제공하기도 했다. 운전대로 게임을 조작하는 방식으로, 차가 주차돼 있을 경우에만 즐길 수 있다.이번 영상 콘텐츠는 주행 중에도 감상할 수 있도록 하는 방안을 고려하고 있다. 테슬라 측은 규제당국이 자율주행에 대해 완전히 승인하면 차량이 움직일 때에도 승객이 동영상을 즐길 수 있을 것이라고 설명했다.하지만 아직까지 자율주행차의 안전에 대한 우려는 완전히 걷혀지지 않은 상황이다. 지난 2017년 차량공유 서비스 우버의 자율주행 시범차량이 보행자와 충돌한 사고가 발생한 바 있다. 게다가 당시 시험 운전자는 디즈니의 동영상 스트리밍 서비스인 \'훌루\'를 이용하고 있던 것으로 밝혀졌다.이민우 기자 letzwin@asiae.co.kr<ⓒ경제를 보는 눈, 세계를 보는 창 아시아경제 무단전재 배포금지>'

In [ ]:
#네이버금융종목토론방게시글 크롤링

from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
from datetime import date, timedelta, datetime
from selenium import webdriver
from bs4 import BeautifulSoup
import dask.dataframe as dd
from random import randint
from time import sleep
from glob import glob
import pandas as pd
import requests
import shutil
import time
import math
import os
import re
from tqdm import tqdm

# path
dir_path='C:/Users/111/naver_stock_crowling'
# 크롬드라이버 path
path = "C:/Users/111/Documents/chromedriver_win32/chromedriver.exe"
driver = webdriver.Chrome(path)
sleep(randint(10, 20) / 10)
driver.get("http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020101")
sleep(randint(10, 20) / 2)
driver.find_element_by_css_selector('button.CI-MDI-UNIT-DOWNLOAD').click()
sleep(randint(10, 20) / 10)
driver.find_elements_by_xpath("//div[@class='filedown_wrap']/div[@data-type='csv']")[0].click()
sleep(randint(10, 20) / 10)
stock_code=glob('C:/Users/111/Downloads/data*.csv')
codelist_path=dir_path+'/codelist/codelist_'+stock_code[0].split('\\')[-1].split('_')[-1]
if len(stock_code)==1:
    shutil.move(stock_code[0],codelist_path)
    
else :
    print("다운로드 폴더를 확인해주세요")
#    break

#C:\Users\111\naver_stock_crowling
#shutil.move(src + filename, dir + filename)


codelist=pd.read_csv(codelist_path,encoding='euc-kr')


for i in tqdm(range(len(list(codelist['종목코드'])))):
    final_df=pd.DataFrame()

    S_code=list(codelist['종목코드'])[i]
    S_name=list(codelist['종목명'])[i]
    driver.get("https://finance.naver.com/item/board.nhn?code="+S_code)

    for Num in range(0,int(str(BeautifulSoup(driver.page_source,'html.parser').select('td.pgRR')).split('">')[1].split('=')[-1])+1):
        driver.get("https://finance.naver.com/item/board.nhn?code="+S_code+'&page='+str(Num))


    #    if not os.path.isdir(dir_path+'/data/'+S_name): 
    #        os.makedirs(dir_path+'/data/'+S_name)
    #    debate_list=glob(dir_path+'/data/'+S_name+'/*')

        texts=str(BeautifulSoup(driver.page_source,'html.parser').select('table.type2'))
        texts=texts.split('onmouseover')[1:]
        time=list(map(lambda x:x.split('</span>')[0].split('>')[-1],texts))
        url=list(map(lambda x:'https://finance.naver.com'+re.sub("amp;","",x.split('href="')[1].split('" onclick')[0]),texts))
        title=list(map(lambda x:"".join(x.split('title="')[1:]).split('"')[0],texts))
        writer=list(map(lambda x:re.sub('[\n\t]',"",x.split('title="')[-1].split('<td class="p11">')[1].split('</td>')[0]),texts))
        view=list(map(lambda x:x.split('title="')[-1].split('<td class="p11">')[1].split('</span')[0].split(">")[-1],texts))
        like=list(map(lambda x:x.split('</strong>')[0].split('>')[-1],texts))
        unlike=list(map(lambda x:x.split('</strong>')[1].split('>')[-1],texts))



        mk_df=pd.DataFrame({"date":time,"url":url,"title":title,"writer":writer,"view":view,"like":like,"unlike":unlike})


        main_texts=[]
        comment=[]
        crowlingtime=[]

        for url in list(mk_df['url']):
            crowlingtime.append(str(datetime.now()))
            try:
                sleep(randint(10, 20) / 10)
                driver.get(url)
                while len(driver.page_source.split('댓글 로딩에 실패하였습니다.'))==2:
                    sleep(randint(10, 20) / 10)
                    driver.find_element_by_css_selector('span.u_cbox_txt_refresh').click()

                main_text_list=str(BeautifulSoup(driver.page_source,'html.parser').select('div.scr01')[0])
                main_texts.append("".join(re.sub("</div>\n</div>","",re.sub('\n<br/>',' ',main_text_list)).split('>')[2:]))

                users=list(map(lambda x:str(x).split('>')[1].split('<')[0],BeautifulSoup(driver.page_source,'html.parser').select('span.u_cbox_nick')))
                ret=list(map(lambda x:str(x).split('>')[1].split('<')[0],BeautifulSoup(driver.page_source,'html.parser').select('span.u_cbox_contents')))
                time=list(map(lambda x:str(x).split('>')[1].split('<')[0],BeautifulSoup(driver.page_source,'html.parser').select('span.u_cbox_date')))
                comment.append("@SPLIT@".join(list(map(lambda x,y,z:x+'_'+y+'_'+z,users,ret,time))))
            except:
                main_texts.append("Error")
                comment.append("Error")

        merge_df=pd.concat([mk_df,pd.DataFrame({'crowlingtime':crowlingtime,'main_texts':main_texts,'comment':comment})],axis=1)    
        final_df=pd.concat([final_df,merge_df]).reset_index(drop=True)


    
    final_df.to_csv(dir_path+'/data/'+S_code+'.csv',encoding="utf-8-sig")
#texts=re.sub("<th>","",texts)
#re.sub("</th>","",texts)
          
          